In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#df_mc = pd.read_pickle(Path("out") / "df_mc_adam_bfgs.pkl")
#df_mc = pd.read_pickle(Path("out") / "df_mc_adam_bfgs_above_1000.pkl")
df_full = pd.read_pickle(Path("out") / "df_mc_full.pkl")
df_red = pd.read_pickle(Path("out") / "df_mc_hypernet.pkl")
df_mc = pd.concat((df_full, df_red), ignore_index=True)

df_full_time = pd.read_pickle(Path("out") / "df_mc_full_time.pkl")

In [ ]:
rms_y = 66.69 # 1e5 rmse of y 
fit_lin = 77.17 # fit of a linear baseline obtained with MATLAB's sysid toolbox
rmse_lin = (1 - fit_lin/100.0) * rms_y # rmse of a linear baseline
rmse_lin/1e5

In [ ]:
df_mc["length"] = df_mc["length"].astype(int)
df_mc["fit"] = np.maximum(df_mc["fit"], 0.0)

df_mc["rmse"] = (1 - df_mc["fit"]/100.0) * rms_y
#df_mc = df_mc[df_mc.length < 2500]
df_mc = df_mc[df_mc.model != "full (adam)"]
#df_mc = df_mc[df_mc.model != "full"]
df_mc["model"] = df_mc["model"].str.replace("full (bfgs)", "full")

In [ ]:
model_labels = {
    "full": "full parameterization",
    "reduced": "reduced parameterization",
    # Add more mappings as needed
}
model_order = ["full", "reduced"]

plt.figure(figsize=(8, 4))
ax = sns.boxplot(df_mc, x="length", y="fit", hue="model")#, hue_order=["full", "reduced"])
plt.tight_layout()  # This adjusts the plot to ensure the legend fits
plt.xticks(rotation=45)
ax.set_xlabel("Training sequence length (-)")
ax.set_ylabel("Test FIT (%)")
ax.axhline(y=fit_lin, color="black", linestyle="dashed", alpha=0.5, label="linear baseline")
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.legend(loc='lower right')#, borderaxespad=0)
plt.ylim([-5, 100])
plt.tight_layout()
#plt.savefig(Path("fig")/"boxplot.pdf")
#plt.savefig(Path("fig")/"boxplot.png")

In [ ]:
plt.figure()
plt.title("Training time (hours)")
plt.plot(df_full_time.length, df_full_time.time/60/24, "*", label="full (adam + BFGS)")
#plt.plot(df_full_time.length, df_full_time.time_adam/60/24, "*", label="full (adam)")
plt.xlabel("Training sequence length (-)")
plt.ylabel("Time (hours)")
plt.legend();

In [ ]:
df_stats = df_mc.groupby(["model", "length"]).agg(["mean", "median", "std"])

In [ ]:
plt.figure()
#plt.plot(df_stats.loc["full"]["fit"]["mean"])
plt.fill_between(df_stats.loc["full"]["fit"]["mean"].index, df_stats.loc["full"]["fit"]["mean"] - df_stats.loc["full"]["fit"]["std"], df_stats.loc["full"]["fit"]["mean"] + df_stats.loc["full"]["fit"]["std"], alpha=0.3)
plt.fill_between(df_stats.loc["reduced"]["fit"]["mean"].index, df_stats.loc["reduced"]["fit"]["mean"] - df_stats.loc["reduced"]["fit"]["std"], df_stats.loc["reduced"]["fit"]["mean"] + df_stats.loc["reduced"]["fit"]["std"], alpha=0.3)

#plt.plot(df_stats.loc["reduced"]["fit"]["mean"])